# NLP Assignment 2

**Authors:** Scott Hallauer (HLLSCO001) and Steve Wang (WNGSHU003)

**Date:** 21 June 2021


Please see the associated README.md file for instructions on how to run this notebook.

## Set-Up Environment



Import all required libraries.

In [101]:
import urllib
import re
import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import ExponentialLR

Set the model's training hyperparameters.


In [102]:
# Manual parameters
NGRAM_SIZE = 6
EMBEDDING_DIM = 10
BATCH_SIZE = 64
INITIAL_LEARNING_RATE = 0.1

# Calculated parameters
CONTEXT_SIZE = NGRAM_SIZE-1

Set the URLs for where the isiXhosa datasets are stored.

In [103]:
train_url = "https://raw.githubusercontent.com/scotthallauer/nlp-feedforward-nn/main/xh-data/nchlt_text.xh.train"
valid_url = "https://raw.githubusercontent.com/scotthallauer/nlp-feedforward-nn/main/xh-data/nchlt_text.xh.valid"
test_url = "https://raw.githubusercontent.com/scotthallauer/nlp-feedforward-nn/main/xh-data/nchlt_text.xh.test"

## Prepare Datasets

Define helper functions for processing the raw datasets.

In [104]:
def load_dataset(url):
  dataset = []
  with urllib.request.urlopen(url) as data:
    for line in data:
      input_sentence = line.decode().split()
      output_sentence = []
      for input_word in input_sentence:
        output_word = input_word.lower()
        output_word = re.sub(r"(\.|\,|\:|\?|\)|\"|\-|\!|\/|\')+$", "", output_word)
        output_word = re.sub(r"^(\(|\"|\-|\/|\')+", "", output_word)
        if len(output_word) > 0:
          output_sentence.append(output_word)
      dataset.append(output_sentence)
  return dataset

def prepare_train_dataset(dataset):
  word_counts = {}
  for sentence in dataset:
    for word in sentence:
      if word not in word_counts:
        word_counts[word] = 1
      else:
        word_counts[word] += 1
  for i in range(len(dataset)):
    for j in range(len(dataset[i])):
      if word_counts[dataset[i][j]] == 1:
        dataset[i][j] = "<unk>"
  return dataset

def prepare_eval_dataset(dataset, train_word_index):
  for i in range(len(dataset)):
    for j in range(len(dataset[i])):
      if dataset[i][j] not in train_word_index:
        dataset[i][j] = "<unk>"
  return dataset

def get_word_index(sentences):
  word_index = {}
  index = 0
  for sentence in sentences:
    for word in sentence:
      if word not in word_index:
        word_index[word] = index
        index += 1
  return word_index

def get_vocab(word_index):
  vocab = []
  for i in range(len(word_index)):
    vocab.append("");
  for word in word_index:
    vocab[word_index[word]] = word
  return vocab

def get_ngrams(sentences, n):
  ngrams = []
  for sentence in sentences:
    if len(sentence) >= n:
      for i in range(len(sentence)-(n-1)):
        ngrams.append((sentence[i:i+(n-1)], sentence[i+(n-1)]))
  return ngrams

Define the XhosaTextTrainDataset and XhosaTextEvalDataset classes as a subclasses of PyTorch's Dataset class.

In [105]:
class XhosaTextTrainDataset(Dataset):
  def __init__(self, dataset_url, ngram_size):
    self.sentences = prepare_train_dataset(load_dataset(dataset_url))
    self.word_index = get_word_index(self.sentences)
    self.vocab = get_vocab(self.word_index)
    self.ngrams = get_ngrams(self.sentences, ngram_size)
    self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

  def __len__(self):
    return len(self.ngrams)

  def __getitem__(self, idx):
    context = []
    for word in self.ngrams[idx][0]:
      context.append(self.word_index[word])
    target = self.word_index[self.ngrams[idx][1]]
    return torch.tensor(context, dtype=torch.long, device=torch.device(self.device)), torch.tensor(target, dtype=torch.long, device=torch.device(self.device))

  def vocab_size(self):
    return len(self.vocab)

  def get_word_index(self):
    return self.word_index

In [106]:
class XhosaTextEvalDataset(Dataset):
  def __init__(self, dataset_url, ngram_size, train_dataset):
    self.word_index = train_dataset.get_word_index()
    self.sentences = prepare_eval_dataset(load_dataset(dataset_url), self.word_index)
    self.vocab = get_vocab(self.word_index)
    self.ngrams = get_ngrams(self.sentences, ngram_size)
    self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

  def __len__(self):
    return len(self.ngrams)

  def __getitem__(self, idx):
    context = []
    for word in self.ngrams[idx][0]:
      context.append(self.word_index[word])
    target = self.word_index[self.ngrams[idx][1]]
    return torch.tensor(context, dtype=torch.long, device=torch.device(self.device)), torch.tensor(target, dtype=torch.long, device=torch.device(self.device))

Download and process all three isiXhosa datasets (training, validation and testing) in preparation for training and evaluating the model.

In [107]:
# Training dataset
train_dataset = XhosaTextTrainDataset(
  dataset_url=train_url, 
  ngram_size=NGRAM_SIZE
)
print(f"Training Dataset: {train_dataset.__len__()} ngrams.")

# Validation dataset
valid_dataset = XhosaTextEvalDataset(
  dataset_url=valid_url, 
  ngram_size=NGRAM_SIZE,
  train_dataset=train_dataset
)
print(f"Validation Dataset: {valid_dataset.__len__()} ngrams.")

# Testing dataset
test_dataset = XhosaTextEvalDataset(
  dataset_url=test_url, 
  ngram_size=NGRAM_SIZE,
  train_dataset=train_dataset
)
print(f"Testing Dataset: {test_dataset.__len__()} ngrams.")

Training Dataset: 655570 ngrams.
Validation Dataset: 36073 ngrams.
Testing Dataset: 36599 ngrams.


Prepare dataloaders for each dataset to efficiently shuffle and batch data for training.

In [108]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

## Define Neural Network Model



Define the structure of the custom neural network model. This model has an input embedding layer, a hidden layer and an output layer. Dropout with 0.25 probability is applied between the layers.

In [109]:
# source: https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html
class NGramLanguageModeler(nn.Module):
  def __init__(self, vocab_size, embedding_dim, context_size, batch_size):
    super(NGramLanguageModeler, self).__init__()
    self.embedding_dim = embedding_dim
    self.context_size = context_size
    self.batch_size = batch_size
    self.embeddings = nn.Embedding(vocab_size, embedding_dim)
    self.linear1 = nn.Linear(context_size * embedding_dim, 128)
    self.linear2 = nn.Linear(128, vocab_size)
    self.dropout = nn.Dropout(0.25)
  
  def forward(self, inputs):
    embeds = self.embeddings(inputs).view((len(inputs), self.context_size * self.embedding_dim))
    out = self.dropout(embeds)
    out = F.relu(self.linear1(out))
    out = self.dropout(out)
    out = self.linear2(out)
    log_probs = F.log_softmax(out, dim=1)
    return log_probs

## Train Neural Network Model

Finally, the neural network model is trained with the training dataset using the specified hyperparameters. The final evaluation is completed using the test dataset.

In [110]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_log_interval = 200

model = NGramLanguageModeler(train_dataset.vocab_size(), EMBEDDING_DIM, CONTEXT_SIZE, BATCH_SIZE).to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=INITIAL_LEARNING_RATE)
scheduler1 = ReduceLROnPlateau(optimizer, patience=1)
scheduler2 = ExponentialLR(optimizer, gamma=0.9)

for epoch_i in range(5):

  print('Start Training')

  # Start timing epoch
  epoch_start_time = datetime.datetime.now()

  # TRAINING PHASE
  model.train()
  total_batch_elapsed_time_ms = 0
  total_batch_loss = 0
  total_batch_ppl = 0

  for batch_i, (train_context, train_target) in enumerate(train_dataloader):

    # Start timing batch
    batch_start_time = datetime.datetime.now()

    # Zero gradients from old input
    model.zero_grad()

    # Forward pass
    log_probs = model(train_context)

    # Get loss
    loss = loss_function(log_probs, train_target)
    total_batch_loss += loss.item()

    # Get perplexity
    ppl = torch.exp(loss)
    total_batch_ppl += ppl.item()

    # Backward propagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Stop timing batch
    batch_end_time = datetime.datetime.now()
    batch_elapsed_time = (batch_end_time - batch_start_time)
    batch_elapsed_time_ms = batch_elapsed_time.total_seconds() * 1000
    total_batch_elapsed_time_ms += batch_elapsed_time_ms

    # Log training stats in batch intervals
    if ((batch_i+1) % batch_log_interval == 0):
      print("| epoch {} | {}/ {} batches | lr {:.4f} | ms/batch {:.2f} | loss {:.2f} | ppl {:.2f}".format(epoch_i+1, batch_i+1, len(train_dataloader), optimizer.param_groups[0]['lr'], total_batch_elapsed_time_ms/batch_log_interval, total_batch_loss/batch_log_interval, total_batch_ppl/batch_log_interval))
      total_batch_elapsed_time_ms = 0
      total_batch_loss = 0
      total_batch_ppl = 0

  # VALIDATION PHASE
  model.eval()
  total_valid_loss = 0
  total_valid_ppl = 0
  with torch.no_grad():
    for batch_i, (valid_context, valid_target) in enumerate(valid_dataloader):

      # Forward pass
      log_probs = model(valid_context)

      # Get loss
      loss = loss_function(log_probs, valid_target)
      total_valid_loss += loss.item()

      # Get perplexity
      ppl = torch.exp(loss)
      total_valid_ppl += ppl.item()

  # Stop timing epoch
  epoch_end_time = datetime.datetime.now()
  epoch_elapsed_time = (epoch_end_time - epoch_start_time)
  epoch_elapsed_time_s = epoch_elapsed_time.total_seconds()
  
  # Update learning rate
  scheduler1.step(total_valid_loss/len(valid_dataloader))
  scheduler2.step()

  # Log validation stats at end of each epoch
  print("-------------------------------------------------------------------")
  print("| end of epoch {} | time: {:.2f}s | valid loss {:.2f} | valid ppl {:.2f}".format(epoch_i+1, epoch_elapsed_time_s, total_valid_loss/len(valid_dataloader), total_valid_ppl/len(valid_dataloader)))
  print("-------------------------------------------------------------------")

# TESTING PHASE
model.eval()
total_test_loss = 0
total_test_ppl = 0

with torch.no_grad():
  for batch_i, (test_context, test_target) in enumerate(test_dataloader):

    # Forward pass
    log_probs = model(test_context)

    # Get loss
    loss = loss_function(log_probs, test_target)
    total_test_loss += loss.item()

    # Get perplexity
    ppl = torch.exp(loss)
    total_test_ppl += ppl.item()

# Log testing stats at end of training
print("=========================================================================================")
print("| End of training | test loss {:.2f} | test ppl {:.2f}".format(total_test_loss/len(test_dataloader), total_test_ppl/len(test_dataloader)))
print("=========================================================================================")

Start Training
| epoch 1 | 200/ 10244 batches | lr 0.1000 | ms/batch 2.19 | loss 10.63 | ppl 43091.37
| epoch 1 | 400/ 10244 batches | lr 0.1000 | ms/batch 2.10 | loss 10.31 | ppl 31248.50
| epoch 1 | 600/ 10244 batches | lr 0.1000 | ms/batch 2.04 | loss 10.14 | ppl 26276.74
| epoch 1 | 800/ 10244 batches | lr 0.1000 | ms/batch 2.04 | loss 9.98 | ppl 22621.13
| epoch 1 | 1000/ 10244 batches | lr 0.1000 | ms/batch 2.04 | loss 9.88 | ppl 20429.69
| epoch 1 | 1200/ 10244 batches | lr 0.1000 | ms/batch 2.03 | loss 9.79 | ppl 18733.42
| epoch 1 | 1400/ 10244 batches | lr 0.1000 | ms/batch 2.05 | loss 9.69 | ppl 16897.98
| epoch 1 | 1600/ 10244 batches | lr 0.1000 | ms/batch 2.04 | loss 9.59 | ppl 15586.08
| epoch 1 | 1800/ 10244 batches | lr 0.1000 | ms/batch 2.06 | loss 9.50 | ppl 14022.23
| epoch 1 | 2000/ 10244 batches | lr 0.1000 | ms/batch 2.04 | loss 9.45 | ppl 13596.24
| epoch 1 | 2200/ 10244 batches | lr 0.1000 | ms/batch 2.04 | loss 9.38 | ppl 12487.71
| epoch 1 | 2400/ 10244 batch